In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score

import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_classification



data_train = pd.read_csv("training.csv")


In [3]:
data_train

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,NaN,MEDICAID,CA,924,84,F,NaN,C50919,Malignant neoplasm of unsp site of unspecified...,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,White,COMMERCIAL,CA,926,45,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
4,817482,NaN,COMMERCIAL,ID,836,55,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12901,674178,White,NaN,OH,436,50,F,32.11,C50411,Malig neoplm of upper-outer quadrant of right ...,...,17.400000,23.600000,0.864706,19.841176,6.300000,6.247059,38.753055,8.068682,21.140731,1
12902,452909,NaN,COMMERCIAL,CA,945,50,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,11.243210,7.837037,5.411250,34.700000,3.845679,5.671605,36.469947,6.265266,10.728732,1
12903,357486,NaN,COMMERCIAL,CA,926,61,F,29.24,C50912,Malignant neoplasm of unspecified site of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,1
12904,935417,NaN,NaN,NY,112,37,F,31.00,1749,"Malignant neoplasm of breast (female), unspeci...",...,10.194737,18.642105,14.173684,42.502632,6.392105,1.755263,37.722740,7.879795,27.496367,0


In [4]:
columns_to_filter = data_train.columns[13:]
count_na = pd.DataFrame(data_train.isna().sum())


In [5]:
with pd.option_context('display.max_rows', None,):
    print(count_na)

                                           0
patient_id                                 0
patient_race                            6385
payer_type                              1803
patient_state                             51
patient_zip3                               0
patient_age                                0
patient_gender                             0
bmi                                     8965
breast_cancer_diagnosis_code               0
breast_cancer_diagnosis_desc               0
metastatic_cancer_diagnosis_code           0
metastatic_first_novel_treatment       12882
metastatic_first_novel_treatment_type  12882
Region                                    52
Division                                  52
population                                 1
density                                    1
age_median                                 1
age_under_10                               1
age_10_to_19                               1
age_20s                                    1
age_30s   

In [8]:
data_train_dropna = data_train.dropna(subset = columns_to_filter)
data_train_dropna

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,NaN,MEDICAID,CA,924,84,F,NaN,C50919,Malignant neoplasm of unsp site of unspecified...,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,White,COMMERCIAL,CA,926,45,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
4,817482,NaN,COMMERCIAL,ID,836,55,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12901,674178,White,NaN,OH,436,50,F,32.11,C50411,Malig neoplm of upper-outer quadrant of right ...,...,17.400000,23.600000,0.864706,19.841176,6.300000,6.247059,38.753055,8.068682,21.140731,1
12902,452909,NaN,COMMERCIAL,CA,945,50,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,11.243210,7.837037,5.411250,34.700000,3.845679,5.671605,36.469947,6.265266,10.728732,1
12903,357486,NaN,COMMERCIAL,CA,926,61,F,29.24,C50912,Malignant neoplasm of unspecified site of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,1
12904,935417,NaN,NaN,NY,112,37,F,31.00,1749,"Malignant neoplasm of breast (female), unspeci...",...,10.194737,18.642105,14.173684,42.502632,6.392105,1.755263,37.722740,7.879795,27.496367,0


In [9]:
columns = ['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'patient_id', 'patient_zip3','patient_gender' ]
data_train_dropna_part2 = data_train_dropna.drop(columns, axis=1)


# Separate categorical and numerical columns
categorical_columns = ['patient_race', 'payer_type', 'patient_state', 'breast_cancer_diagnosis_code', 
                       'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'Division']
numerical_columns = data_train_dropna_part2.columns.difference(categorical_columns)

# Fill missing values with mode for categorical columns and mean for numerical columns
for col in categorical_columns:
    mode = data_train_dropna_part2[col].mode()[0]
    data_train_dropna_part2[col].fillna(mode, inplace=True)

for col in numerical_columns:
    mean = data_train_dropna_part2[col].mean()
    data_train_dropna_part2[col].fillna(mean, inplace=True)

# Print the modified DataFrame
print(data_train_dropna_part2)


      patient_race  payer_type patient_state  patient_age        bmi  \
0            White    MEDICAID            CA           84  28.985495   
1            White  COMMERCIAL            CA           62  28.490000   
2            White  COMMERCIAL            TX           43  38.090000   
3            White  COMMERCIAL            CA           45  28.985495   
4            White  COMMERCIAL            ID           55  28.985495   
...            ...         ...           ...          ...        ...   
12901        White  COMMERCIAL            OH           50  32.110000   
12902        White  COMMERCIAL            CA           50  28.985495   
12903        White  COMMERCIAL            CA           61  29.240000   
12904        White  COMMERCIAL            NY           37  31.000000   
12905     Hispanic    MEDICAID            FL           29  28.985495   

      breast_cancer_diagnosis_code  \
0                           C50919   
1                           C50411   
2                    

In [10]:
count_na_2 = pd.DataFrame(data_train_dropna_part2.isna().sum())

with pd.option_context('display.max_rows', None,):
    print(count_na_2)

                                  0
patient_race                      0
payer_type                        0
patient_state                     0
patient_age                       0
bmi                               0
breast_cancer_diagnosis_code      0
breast_cancer_diagnosis_desc      0
metastatic_cancer_diagnosis_code  0
Region                            0
Division                          0
population                        0
density                           0
age_median                        0
age_under_10                      0
age_10_to_19                      0
age_20s                           0
age_30s                           0
age_40s                           0
age_50s                           0
age_60s                           0
age_70s                           0
age_over_80                       0
male                              0
female                            0
married                           0
divorced                          0
never_married               

In [11]:
columns_object = data_train_dropna_part2.dtypes[data_train_dropna_part2.dtypes == "object"]
columns_object.index

Index(['patient_race', 'payer_type', 'patient_state',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],
      dtype='object')

In [12]:

# Create dummy variables for categorical columns
data_train_dropna_part3  = pd.get_dummies(data_train_dropna_part2, columns=['patient_race','payer_type', 'patient_state',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],  prefix = ['patient_race','payer_type', 'patient_state',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'], prefix_sep='_')
data_train_dropna_part3

,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,...,Region_West,Division_East North Central,Division_East South Central,Division_Middle Atlantic,Division_Mountain,Division_New England,Division_Pacific,Division_South Atlantic,Division_West North Central,Division_West South Central
0,84,28.985495,31437.75000,1189.562500,30.642857,16.014286,15.542857,17.614286,14.014286,11.614286,...,True,False,False,False,False,False,True,False,False,False
1,62,28.490000,39121.87879,2295.939394,38.200000,11.878788,13.354545,14.230303,13.418182,13.333333,...,True,False,False,False,False,False,True,False,False,False
2,43,38.090000,21996.68333,626.236667,37.906667,13.028333,14.463333,12.531667,13.545000,12.860000,...,False,False,False,False,False,False,False,False,False,True
3,45,28.985495,32795.32558,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,...,True,False,False,False,False,False,True,False,False,False
4,55,28.985495,10886.26000,116.886000,43.473469,10.824000,13.976000,9.492000,10.364000,12.600000,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12901,50,32.110000,19413.05882,1196.805882,36.911765,12.876471,13.435294,14.394118,12.705882,11.694118,...,False,True,False,False,False,False,False,False,False,False
12902,50,28.985495,30153.87952,976.289157,42.135802,10.753086,12.714815,11.725926,13.101235,12.817284,...,True,False,False,False,False,False,True,False,False,False
12903,61,29.240000,32795.32558,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,...,True,False,False,False,False,False,True,False,False,False
12904,37,31.000000,71374.13158,17326.407890,36.476316,12.986842,11.318421,14.971053,17.255263,12.631579,...,False,False,False,True,False,False,False,False,False,False


In [13]:
data_train_dropna_part3['bmi'] = data_train_dropna_part3['bmi'].replace('UNKNOWN', 0)
data_train_dropna_part3

,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,...,Region_West,Division_East North Central,Division_East South Central,Division_Middle Atlantic,Division_Mountain,Division_New England,Division_Pacific,Division_South Atlantic,Division_West North Central,Division_West South Central
0,84,28.985495,31437.75000,1189.562500,30.642857,16.014286,15.542857,17.614286,14.014286,11.614286,...,True,False,False,False,False,False,True,False,False,False
1,62,28.490000,39121.87879,2295.939394,38.200000,11.878788,13.354545,14.230303,13.418182,13.333333,...,True,False,False,False,False,False,True,False,False,False
2,43,38.090000,21996.68333,626.236667,37.906667,13.028333,14.463333,12.531667,13.545000,12.860000,...,False,False,False,False,False,False,False,False,False,True
3,45,28.985495,32795.32558,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,...,True,False,False,False,False,False,True,False,False,False
4,55,28.985495,10886.26000,116.886000,43.473469,10.824000,13.976000,9.492000,10.364000,12.600000,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12901,50,32.110000,19413.05882,1196.805882,36.911765,12.876471,13.435294,14.394118,12.705882,11.694118,...,False,True,False,False,False,False,False,False,False,False
12902,50,28.985495,30153.87952,976.289157,42.135802,10.753086,12.714815,11.725926,13.101235,12.817284,...,True,False,False,False,False,False,True,False,False,False
12903,61,29.240000,32795.32558,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,...,True,False,False,False,False,False,True,False,False,False
12904,37,31.000000,71374.13158,17326.407890,36.476316,12.986842,11.318421,14.971053,17.255263,12.631579,...,False,False,False,True,False,False,False,False,False,False


In [14]:

from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
data_train_dropna_part4 =  pd.DataFrame(scaler.fit_transform(data_train_dropna_part3), columns=data_train_dropna_part3.columns)

data_train_dropna_part4

,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,...,Region_West,Division_East North Central,Division_East South Central,Division_Middle Atlantic,Division_Mountain,Division_New England,Division_Pacific,Division_South Atlantic,Division_West North Central,Division_West South Central
0,0.904110,0.211063,0.435437,0.056145,0.068802,0.866679,0.396745,0.434950,0.420716,0.423201,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.602740,0.204085,0.544064,0.108404,0.362911,0.534683,0.302667,0.309035,0.390576,0.582980,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.342466,0.339296,0.301973,0.029537,0.351495,0.626968,0.350335,0.245829,0.396988,0.538986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.369863,0.211063,0.454629,0.089523,0.544714,0.389589,0.250269,0.246069,0.342346,0.519467,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.506849,0.211063,0.144910,0.005478,0.568144,0.450005,0.329384,0.132726,0.236154,0.514820,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12817,0.438356,0.255070,0.265449,0.056487,0.312775,0.614776,0.306139,0.315130,0.354561,0.430621,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12818,0.438356,0.211063,0.417288,0.046071,0.516085,0.444312,0.275165,0.215848,0.374551,0.535016,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12819,0.589041,0.214648,0.454629,0.089523,0.544714,0.389589,0.250269,0.246069,0.342346,0.519467,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12820,0.260274,0.239437,1.000000,0.818356,0.295829,0.623637,0.215132,0.336597,0.584583,0.517755,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
pd.set_option('display.max_columns', None)


In [16]:
data_train_dropna_part4_x = data_train_dropna_part4.drop("DiagPeriodL90D", axis = 1).reset_index(drop = True)

data_train_dropna_part4_y = data_train_dropna_part4[["DiagPeriodL90D"]]

data_train_dropna_part4_x

,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,patient_race_Asian,patient_race_Black,patient_race_Hispanic,patient_race_Other,patient_race_White,payer_type_COMMERCIAL,payer_type_MEDICAID,payer_type_MEDICARE ADVANTAGE,patient_state_AL,patient_state_AR,patient_state_AZ,patient_state_CA,patient_state_CO,patient_state_CT,patient_state_DC,patient_state_DE,patient_state_FL,patient_state_GA,patient_state_IA,patient_state_ID,patient_state_IL,patient_state_IN,patient_state_KS,patient_state_KY,patient_state_LA,patient_state_MA,patient_state_MD,patient_state_MI,patient_state_MN,patient_state_MO,patient_state_MS,patient_state_MT,patient_state_NC,patient_state_ND,patient_state_NE,patient_state_NH,patient_state_NJ,patient_state_NM,patient_state_NV,patient_state_NY,patient_state_OH,patient_state_OK,patient_state_OR,patient_state_PA,patient_state_RI,patient_state_SC,patient_state_SD,patient_state_TN,patient_state_TX,patient_state_UT,patient_state_VA,patient_state_WA,patient_state_WI,patient_state_WV,patient_state_WY,breast_cancer_diagnosis_code_1741,breast_cancer_diagnosis_code_1742,breast_cancer_diagnosis_code_1743,breast_cancer_diagnosis_code_1744,breast_cancer_diagnosis_code_1745,breast_cancer_diagnosis_code_1746,breast_cancer_diagnosis_code_1748,breast_cancer_diagnosis_code_1749,breast_cancer_diagnosis_code_1759,breast_cancer_diagnosis_code_19881,breast_cancer_diagnosis_code_C50,breast_cancer_diagnosis_code_C5001,breast_cancer_diagnosis_code_C50011,breast_cancer_diagnosis_code_C50012,breast_cancer_diagnosis_code_C50019,breast_cancer_diagnosis_code_C50021,breast_cancer_diagnosis_code_C5011,breast_cancer_diagnosis_code_C50111,breast_cancer_diagnosis_code_C50112,breast_cancer_diagnosis_code_C50119,breast_cancer_diagnosis_code_C5021,breast_cancer_diagnosis_code_C50211,breast_cancer_diagnosis_code_C50212,breast_cancer_diagnosis_code_C50219,breast_cancer_diagnosis_code_C5031,breast_cancer_diagnosis_code_C50311,breast_cancer_diagnosis_code_C50312,breast_cancer_diagnosis_code_C50319,breast_cancer_diagnosis_code_C5041,breast_cancer_diagnosis_code_C50411,breast_cancer_diagnosis_code_C50412,breast_cancer_diagnosis_code_C50419,breast_cancer_diagnosis_code_C50421,breast_cancer_diagnosis_code_C5051,breast_cancer_diagnosis_code_C50511,breast_cancer_diagnosis_code_C50512,breast_cancer_diagnosis_code_C50519,breast_cancer_diagnosis_code_C50611,breast_cancer_diagnosis_code_C50612,breast_cancer_diagnosis_code_C50619,breast_cancer_diagnosis_code_C5081,breast_cancer_diagnosis_code_C50811,breast_cancer_diagnosis_code_C50812,breast_cancer_diagnosis_code_C50819,breast_cancer_diagnosis_code_C509,breast_cancer_diagnosis_code_C5091,breast_cancer_diagnosis_code_C50911,breast_cancer_diagnosis_code_C50912,breast_cancer_diagnosis_code_C50919,breast_cancer_diagnosis_code_C50929,breast_cancer_diagnosis_desc_Malig neoplasm of lower-inner quadrant of left female breast,breast_cancer_diagnosis_desc_Malig neoplasm of lower-inner quadrant of unsp female breast,breast_cancer_diagnosis_desc_Malig neop

In [17]:

X_train, X_test, y_train, y_test = train_test_split(data_train_dropna_part4_x  ,data_train_dropna_part4_y,  stratify = data_train_dropna_part4_y,
                                                    train_size = 0.8, random_state = 20240114)#coba gridsearch tanpa PCA #current best
xgb_tuned = xgb.XGBClassifier(random_state=1)


In [ ]:

# Grid of parameters to choose from

parameters = {
    "n_estimators": [10, 50, 100],
    "subsample":[0.6, 0.8, 1],
    "learning_rate":[0.01, 0.1, 0.5, 1],
    "gamma":[0.01, 0.1, 1, 5],
    "colsample_bytree":[0.5, 0.7, 0.9, 1],
    "alpha":[0, 0.1, 0.5]
}


# Run the grid search
grid_obj = GridSearchCV(xgb_tuned, parameters, scoring="roc_auc",
                        cv=5, n_jobs=-1, verbose=2)

grid_obj = grid_obj.fit(X_train, y_train)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


In [155]:

# Predict probabilities on the test set
y_pred_proba = grid_obj.predict_proba(X_test)

# Compute ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])
print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.7873621109646172


In [156]:


data_test = pd.read_csv("test.csv")

columns = ['patient_id','metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type','patient_zip3','patient_gender' ]
data_test_dropna_part2 = data_test.drop(columns, axis=1)

object_columns = data_test_dropna_part2 .select_dtypes(include=['object']).columns

# Fill NaN values only in object columns
data_test_dropna_part2 [object_columns] = data_test_dropna_part2 [object_columns].fillna('UNKNOWN')

# Identify numeric columns
numeric_columns = data_test_dropna_part2.select_dtypes(include=['float', 'int']).columns
data_test_dropna_part2[numeric_columns]

,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
0,54,NaN,5441.435484,85.620968,40.880328,12.732258,14.088710,10.659677,11.625806,11.208065,15.619355,12.322581,8.409677,3.343548,49.154839,50.845161,55.175806,13.982258,24.266129,6.583871,3.073226,52.980645,66187.22807,1.611290,1.277419,2.645161,3.853226,3.172581,13.275806,12.633871,21.485484,16.717742,15.238710,8.070968,23.309677,33553.43333,84.112903,2064.741935,152749.5370,825.122449,23.895455,12.429032,40.667742,28.959677,11.895161,6.046774,17.941935,35.591379,63.303226,3.406557,10.655357,5.551786,94.793548,0.364516,0.303226,0.119355,0.009677,0.770968,3.630645,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
1,52,NaN,19613.820510,1555.107692,49.107692,8.069231,8.587179,10.684615,11.302564,10.971795,15.823077,15.902564,11.828205,6.815385,49.658974,50.341026,44.800000,17.779487,29.102564,8.310256,2.917105,46.665789,64711.71053,3.873684,2.044737,3.807895,4.239474,4.242105,9.347368,13.018421,17.373684,12.889474,14.442105,14.702632,29.144737,34678.61538,68.673684,8502.230769,265860.6053,1343.394737,34.957895,8.379487,26.558974,30.200000,22.100000,12.764103,34.864103,43.250000,57.035897,5.002632,11.564103,0.005128,78.217949,10.889744,3.453846,0.187179,0.076923,1.841026,5.328205,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
2,61,NaN,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
3,64,NaN,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
4,40,23.00,36054.117650,5294.330882,36.653846,9.761538,11.267692,17.233846,17.441538,13.090769,12.304615,9.407692,5.673846,3.824615,50.510769,49.489231,33.478462,11.301538,50.456923,4.766154,3.442857,55.531746,69266.69355,6.320312,2.953125,6.806250,4.175000,4.125000,7

In [157]:

# Fill NaN values only in numeric columns
data_test_dropna_part2[numeric_columns] = data_test_dropna_part2[numeric_columns].fillna(0)  # You can replace 0 with any value you want
data_test_dropna_part2[numeric_columns]

,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
0,54,0.00,5441.435484,85.620968,40.880328,12.732258,14.088710,10.659677,11.625806,11.208065,15.619355,12.322581,8.409677,3.343548,49.154839,50.845161,55.175806,13.982258,24.266129,6.583871,3.073226,52.980645,66187.22807,1.611290,1.277419,2.645161,3.853226,3.172581,13.275806,12.633871,21.485484,16.717742,15.238710,8.070968,23.309677,33553.43333,84.112903,2064.741935,152749.5370,825.122449,23.895455,12.429032,40.667742,28.959677,11.895161,6.046774,17.941935,35.591379,63.303226,3.406557,10.655357,5.551786,94.793548,0.364516,0.303226,0.119355,0.009677,0.770968,3.630645,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
1,52,0.00,19613.820510,1555.107692,49.107692,8.069231,8.587179,10.684615,11.302564,10.971795,15.823077,15.902564,11.828205,6.815385,49.658974,50.341026,44.800000,17.779487,29.102564,8.310256,2.917105,46.665789,64711.71053,3.873684,2.044737,3.807895,4.239474,4.242105,9.347368,13.018421,17.373684,12.889474,14.442105,14.702632,29.144737,34678.61538,68.673684,8502.230769,265860.6053,1343.394737,34.957895,8.379487,26.558974,30.200000,22.100000,12.764103,34.864103,43.250000,57.035897,5.002632,11.564103,0.005128,78.217949,10.889744,3.453846,0.187179,0.076923,1.841026,5.328205,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
2,61,0.00,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
3,64,0.00,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
4,40,23.00,36054.117650,5294.330882,36.653846,9.761538,11.267692,17.233846,17.441538,13.090769,12.304615,9.407692,5.673846,3.824615,50.510769,49.489231,33.478462,11.301538,50.456923,4.766154,3.442857,55.531746,69266.69355,6.320312,2.953125,6.806250,4.175000,4.1250

In [158]:
data_test_dropna_part2

,patient_race,payer_type,patient_state,patient_age,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,Division,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
0,White,MEDICAID,IN,54,0.00,C50412,Malig neoplasm of upper-outer quadrant of left...,C773,Midwest,East North Central,5441.435484,85.620968,40.880328,12.732258,14.088710,10.659677,11.625806,11.208065,15.619355,12.322581,8.409677,3.343548,49.154839,50.845161,55.175806,13.982258,24.266129,6.583871,3.073226,52.980645,66187.22807,1.611290,1.277419,2.645161,3.853226,3.172581,13.275806,12.633871,21.485484,16.717742,15.238710,8.070968,23.309677,33553.43333,84.112903,2064.741935,152749.5370,825.122449,23.895455,12.429032,40.667742,28.959677,11.895161,6.046774,17.941935,35.591379,63.303226,3.406557,10.655357,5.551786,94.793548,0.364516,0.303226,0.119355,0.009677,0.770968,3.630645,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
1,UNKNOWN,COMMERCIAL,FL,52,0.00,C50912,Malignant neoplasm of unspecified site of left...,C787,South,South Atlantic,19613.820510,1555.107692,49.107692,8.069231,8.587179,10.684615,11.302564,10.971795,15.823077,15.902564,11.828205,6.815385,49.658974,50.341026,44.800000,17.779487,29.102564,8.310256,2.917105,46.665789,64711.71053,3.873684,2.044737,3.807895,4.239474,4.242105,9.347368,13.018421,17.373684,12.889474,14.442105,14.702632,29.144737,34678.61538,68.673684,8502.230769,265860.6053,1343.394737,34.957895,8.379487,26.558974,30.200000,22.100000,12.764103,34.864103,43.250000,57.035897,5.002632,11.564103,0.005128,78.217949,10.889744,3.453846,0.187179,0.076923,1.841026,5.328205,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
2,Hispanic,MEDICAID,CA,61,0.00,C50911,Malignant neoplasm of unsp site of right femal...,C773,West,Pacific,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
3,Hispanic,MEDICARE ADVANTAGE,CA,64,0.00,C50912,Malignant neoplasm of unspecified site of left...,C779,West,Pacific,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.500

In [159]:


# Create dummy variables for categorical columns
data_test_dropna_part3  = pd.get_dummies(data_test_dropna_part2, columns=['patient_race','payer_type', 'patient_state',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],  prefix = ['patient_race','payer_type', 'patient_state',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'], prefix_sep='_')


# fit and transform the data
data_test_dropna_part4 =  pd.DataFrame(scaler.fit_transform(data_test_dropna_part3), columns=data_test_dropna_part3.columns)

data_test_dropna_part4_x = data_test_dropna_part4

data_test_dropna_part4_x  = data_test_dropna_part4_x.reindex(columns=data_train_dropna_part4_x.columns, fill_value=0) 
result = pd.concat([pd.DataFrame(data_test['patient_id']),pd.DataFrame(grid_obj.predict_proba(data_test_dropna_part4_x))[1]], axis = 1).rename(columns = { 1:"DiagPeriodL90D"})

result['patient_id'] = result['patient_id'].astype(int)



In [160]:
result.to_csv("output.csv", index = False)